In [6]:
# monta o drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# define o caminho das pastas - mudar aqui caso precise prof
pastaImagensChroma = '/content/drive/MyDrive/UP/VisaoComputacional/AtividadesAvaliativas/AtvAvaliativa2/src/imgs/'
pastaResultado = '/content/drive/MyDrive/UP/VisaoComputacional/AtividadesAvaliativas/AtvAvaliativa2/src/resultado/'

# lista com todas as imagens - se precisar adicionar ou retirar imagens
list_img = ["corvo.bmp","corvos.bmp","rainha.bmp", "formas.bmp"]

In [42]:
import cv2
import numpy as np

for imagem in list_img:
  # Leitura da imagem
  img = cv2.imread(pastaImagensChroma+imagem)

  # Leitura do background
  arqBackground = 'background.bmp' # mudar o nome aqui caso precise
  background = cv2.imread(pastaImagensChroma+arqBackground)

  # Pega as dimensoes da imagem
  height, width, channels = img.shape

  # Ajustamos o tamanho do background de acordo com a imagem e aplicamos o filtro gaussiano
  background_resize = cv2.resize(background, (width, height))

  # Aplicação do filtro gaussiano no background
  background_blur = cv2.GaussianBlur(background_resize, (5, 5), 0)

  # Conversão para o espaço de cores HSV
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  bg_hsv = cv2.cvtColor(background_blur, cv2.COLOR_BGR2HSV)

  # Definição das faixas de valores HSV para o verde
  #lower_green = np.array([50, 100, 100])
  #upper_green = np.array([70, 255, 255])
  lower_green = np.array([45, 100, 100])
  upper_green = np.array([70, 255, 255])

  # Criação da máscara para isolar as regiões verdes
  mascara = cv2.inRange(hsv, lower_green, upper_green)

  # Binarizando a imagem
  _, mask = cv2.threshold(mascara, 0, 255, cv2.THRESH_BINARY)
  cv2.imwrite(pastaResultado+imagem.replace(".bmp", "/")+imagem.replace(".bmp", "_binarizada.bmp"), mask)
  
  # Suavização das bordas da máscara
  # pra testar com valores diferentes mudar o kernelG
  kernelG = (11,11)
  blur = cv2.GaussianBlur(mask, kernelG, 0)
  cv2.imwrite(pastaResultado+imagem.replace(".bmp", "/")+imagem.replace(".bmp", "_gaussiano.bmp"), blur)

  # pra testar com valores diferentes mudar o kernel
  kernel = np.ones((11,11),np.uint8)
  sobelx = cv2.Sobel(blur, cv2.CV_64F, 1, 0, kernel)
  sobely = cv2.Sobel(blur, cv2.CV_64F, 0, 1, kernel)
  mascara_sobel = cv2.convertScaleAbs(cv2.addWeighted(sobelx, 0.5, sobely, 0.5, 0))
  cv2.imwrite(pastaResultado+imagem.replace(".bmp", "/")+imagem.replace(".bmp", "_sobel.bmp"), mascara_sobel)

  # Aplicação da máscara no background
  masked_bg = cv2.bitwise_and(bg_hsv, bg_hsv, mask=mask)

  # Inversão da máscara para obter as regiões não verdes
  mask_inv = cv2.bitwise_not(mask)

  # Aplicação da máscara invertida na imagem de entrada
  masked_img = cv2.bitwise_and(hsv, hsv, mask=mask_inv)

  # Mesclagem do background e da imagem de entrada
  result_hsv = cv2.add(masked_bg, masked_img)

  # Conversão de volta para o espaço de cores BGR
  result_bgr = cv2.cvtColor(result_hsv, cv2.COLOR_HSV2BGR)

  # Salvar a imagem resultante
  cv2.imwrite(pastaResultado+imagem.replace(".bmp", "/")+imagem, result_bgr)